In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Set data path for the OK and NOK folders
data_path = "mamba/in_test_transformer/"

# Load data from an Excel file
def load_data(file_path):
    return pd.read_excel(file_path)[['Torque (cN·m)', 'Angle (deg)']]

# Pad sequences to the max length in the batch
def pad_sequence(seq, max_len):
    padded_seq = torch.zeros((max_len, seq.size(1)))
    padded_seq[:seq.size(0), :] = seq
    return padded_seq

# Custom Dataset class for OK/NOK files with padding
class ScrewDataset(Dataset):
    def __init__(self, ok_files, nok_files):
        self.data = []
        self.labels = []
        
        # Process OK files
        for file in ok_files:
            df = load_data(data_path + 'OK/' + file)
            self.data.append(torch.tensor(df.values, dtype=torch.float32))
            self.labels.append(0)  # Label 0 for OK
        
        # Process NOK files
        for file in nok_files:
            df = load_data(data_path + 'NOK/' + file)
            self.data.append(torch.tensor(df.values, dtype=torch.float32))
            self.labels.append(1)  # Label 1 for NOK

        # Find the max sequence length
        self.max_len = max(seq.size(0) for seq in self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        padded_data = pad_sequence(self.data[idx], self.max_len)
        return padded_data, self.labels[idx]

# Collect file names from the cloned dataset
ok_files = [f for f in os.listdir(data_path + 'OK') if f.endswith('.xlsx')]
nok_files = [f for f in os.listdir(data_path + 'NOK') if f.endswith('.xlsx')]

# Initialize dataset and dataloader
dataset = ScrewDataset(ok_files, nok_files)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define the Transformer model
class ScrewTransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim, num_layers, num_classes):
        super(ScrewTransformerModel, self).__init__()
        self.transformer = nn.Transformer(
            d_model=input_dim, nhead=num_heads, num_encoder_layers=num_layers, batch_first=True
        )
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.transformer(x, x)
        x = x.mean(dim=1)  # Average over time for classification
        x = self.fc(x)
        return self.classifier(x)

# Model parameters
input_dim = 2  # Torque and Angle
num_heads = 2
hidden_dim = 64
num_layers = 2
num_classes = 2  # OK/NOK

# Initialize model, criterion, and optimizer
model = ScrewTransformerModel(input_dim, num_heads, hidden_dim, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Example inference
sample_data, _ = dataset[0]
model.eval()
with torch.no_grad():
    prediction = model(sample_data.unsqueeze(0))  # Add batch dimension
    print("Predicted OK/NOK:", prediction.argmax(dim=1).item())


ModuleNotFoundError: No module named 'torch._C'